# EPA1361 - Model-Based Decision Making
# Week 3 - Sensitivity analysis

This exercise uses the same predator-prey model we used for the multi-model exercise, focusing on the Python version. As with the other exercise, define a model object for the function below, with the uncertainty ranges provided:

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |

* Sensitivity analysis often focuses on the final values of an outcome at the end of the simulation. However, we can also look at metrics that give us additional information about the behavior of the model over time. Using [the statsmodel library](https://www.statsmodels.org/stable/index.html) and an appropriate sampling design, fit a linear regression model for each of the following indicators. What can we conclude about the behavior of the model, and about the importance of the different inputs?

  * The final values of the _prey_ outcome
  * The mean values of the _prey_ outcome over time, within each experiment
  * The standard deviations of the _prey_ outcome over time, within each experiment
  

* Use the Sobol sampling functionality included in the Workbench to perform experiments with a sample size of N=50, then analyze the results with SALib for the same three indicators. This requires specifying the keyword argument `'uncertainty_sampling'` of perform_experiments. Note that when using Sobol sampling, the meaning of the keyword argument `scenarios` changes a bit. In order to properly estimate Sobol scores as well as interaction effects, you require N * (2D+2) scenarios, where D is the number of uncertain parameters, and N is the value for scenarios passed to `perform_experiments`. Repeat the analysis for larger sample sizes, with N=250 and N=1000. How can we interpret the first-order and total indices? Are these sample sizes sufficient for a stable estimation of the indices? You'll need to use the [get_SALib_problem](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/em_framework/salib_samplers.html) function to convert your Workbench experiments to a problem definition that you can pass to the SALib analysis function. 

* *hint*: sobol is a deterministic sequence of quasi random numbers. Thus, you can run with N=1000 and simply slice for 1:50 and 1:250.

* Use the [Extra-Trees analysis](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html) included in the Workbench to approximate the Sobol total indices, with a suitable sampling design. As a starting point, use an ensemble of 100 trees and a max_features parameter of 0.6, and set the analysis to regression mode. Are the estimated importances stable relative to the sample size and the analysis parameters? How do the results compare to the Sobol indices? For more details on this analysis see [Jaxa-Rozen & Kwakkel (2018)](https://www.sciencedirect.com/science/article/pii/S1364815217311581)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ema_workbench
from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging)

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf
from ema_workbench import (RealParameter, ScalarOutcome, Constant, Model)

def pred_prey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, 
             final_time=365, reps=1):

    #Initial values
    predators = np.zeros((reps, int(final_time/dt)+1))
    prey = np.zeros((reps, int(final_time/dt)+1))
    sim_time = np.zeros((reps, int(final_time/dt)+1))
    
    for r in range(reps):

        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

    #Calculate the time series
    for t in range(0, sim_time.shape[1]-1):

        dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
        dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

        prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
        predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
        sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

ModuleNotFoundError: No module named 'ema_workbench'

In [ ]:
#importing statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Initiating the Model
The cell below initiates the model with the corresponding uncertainties, outcomes and constant in and of the model.

In [ ]:
from ema_workbench import (RealParameter, ScalarOutcome, Constant, Model)

#Creating a model instance
model = Model('predator', function=pred_prey)

#specify uncertainties
model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

model.constants = [Constant('initial_prey', 50),
                   Constant('initial_predators', 20),
                   Constant('dt', 0.25),
                   Constant('final_time', 365),
                   Constant('reps', 1)]

n_scenarios_1 = 50
n_scenarios_2 = 250
n_scenarios_3 = 1000

### Analyzing the Results
In order to analyze all the different results, one function was created where the array which had to be analyzed was given to. One analysis array is given to the function, which will be the array that the sobol function uses to conduct the final analysis.

In [ ]:
def analyze_results(results, variable, analyze_array):
    _, outcomes = results
    problem = get_SALib_problem(model.uncertainties)
    y = outcomes[variable]
    sobol_indices = sobol.analyze(problem, analyze_array)
    sobol_stats = {key: sobol_indices[key] for key in ['ST', 'ST_conf', 'S1',
                                                       'S1_conf']}
    sobol_stats = pd.DataFrame(sobol_stats, index=problem['names'])
    sobol_stats.sort_values(by='ST', ascending=False)
    s2 = pd.DataFrame(sobol_indices['S2'], index=problem['names'],
                      columns=problem['names'])
    s2_conf = pd.DataFrame(sobol_indices['S2_conf'], index=problem['names'],
                           columns=problem['names'])

    return sobol_stats, s2, s2_conf

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)
#Creating a model instance
model = Model('predator', function=pred_prey)

#specify uncertainties
model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

model.constants = [Constant('initial_prey', 50),
                   Constant('initial_predators', 20),
                   Constant('dt', 0.25),
                   Constant('final_time', 365),
                   Constant('reps', 1)]

# Select the number of scenarios that you wish to run. This value must be
# replaced in the perform_experiments function.
n_scenarios_1 = 50
n_scenarios_2 = 250
n_scenarios_3 = 1000

with ema_workbench.SequentialEvaluator(model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios_1, uncertainty_sampling=SOBOL)

# Use the Multi-processing evaluator in pycharm inside the main function, saves a lot of time.
# with ema_workbench.MultiprocessingEvaluator(model) as evaluator:
#     results = evaluator.perform_experiments(n_scenarios_1, uncertainty_sampling=SOBOL)

x, outcomes = results
y = outcomes['prey']
last_values = np.zeros(shape=len(y))
mean_values = np.zeros(shape=len(y))
standard_dev_values = np.zeros(shape=len(y))
for i in range(len(y)):
    last_values[i] = y[i][-1][-1]
    mean_values[i] = y[i][-1].mean()         
    standard_dev_values[i] = y[i][-1].std()  

In [ ]:
sobol_stats, s2, s2_conf = analyze_results(results, 'prey', last_values)
print(sobol_stats)
print(s2)
print(s2_conf)

In [ ]:
sobol_stats, s2, s2_conf = analyze_results(results, 'prey', mean_values)
print(sobol_stats)
print(s2)
print(s2_conf)

In [ ]:
sobol_stats, s2, s2_conf = analyze_results(results, 'prey', standard_dev_values)
print(sobol_stats)
print(s2)
print(s2_conf)

## Extra Trees Analysis
The final analysis, which is the `extra_trees` analysis will be performed using the `get_ex_feature_scores` function in the ema_workbench module.

In [ ]:
x = x.drop(['model', 'policy'], axis=1)
y = np.max(outcomes['prey'], axis=1)

all_scores = []

# Change this to 100, although it increases the time by a lot
for i in range(10):
    print(f"Running {i}")
    indices = np.random.choice(np.arange(0, x.shape[0]), size=x.shape[0])# sobol_stats, s2, s2_conf = analyze_results(results, 'prey', mean_values)
    selected_x = x.iloc[indices, :]
    selected_y = y[indices]
    
    scores = feature_scoring.get_ex_feature_scores(selected_x,
                                                   selected_y,
                                                   max_features=0.6,
                                                   mode=RuleInductionType.REGRESSION)[0]# sobol_stats, s2, s2_conf = analyze_results(results, 'prey', standard_dev_values)
    all_scores.append(scores)# print(sobol_stats)
all_scores = pd.concat(all_scores, axis=1, sort=False)# print(s2)
# print(s2_conf)
sns.boxplot(data=all_scores.T)
plt.show()


What can be seen from the plot above is that the variables are relatively stable compared to their values. The `predation_rate` has one of the lowest absolute values and consequently one of the lowest uncertainties as well. This is a good sign that the model is not very sensitive to changes in the values.